In [1]:
import pandas as pd
import sys

In [2]:
gene_db = pd.read_table("feature_db_part1.txt")

In [ ]:
gene_db.head()

In [ ]:
gene_db

In [3]:
cds_db = pd.read_table("feature_db_part2.txt")

In [ ]:
cds_db

In [4]:
def get_gene_record(identifier,by,source="feature_db_part1.txt"):
    gene_db = pd.read_table(source)
    if by == "gene_id":
        gene_id = identifier
        if sum(gene_db.gene_id==gene_id)<1:
            print ("Not found in our database")
            sys.exit()
        else:
            return gene_db[gene_db.gene_id==gene_id]
    else:
        gene_name = identifier
        if sum(gene_db.gene_name==gene_name)<1:
            print ("Not found in our database")
            sys.exit()
        else:
            return gene_db[gene_db.gene_name==gene_name]        

In [5]:
def fa_parser(fh):
    seq_id = None
    seq=[]
    for line in fh:
        line=line.rstrip()
        if line.startswith(">"):
            if seq_id:
                yield(seq_id,"".join(seq))
            seq_id=line[1:]
            seq=[]
        else:
            seq.append(line)
    if seq_id: yield(seq_id,"".join(seq))      

In [6]:
class Gene:
    def __init__(self,**kwargs):
        self.gene_id= kwargs.get("gene_id",-1)
        self.gene_name=kwargs.get("gene_name",-1)
        legal_arg=["gene_id","gene_name"]
        if not [i for i in kwargs.keys()][0] in legal_arg:
            raise ValueError("must init with gene_id or gene_name")
        else:
            if  self.gene_id != -1:
                record_df = get_gene_record(self.gene_id,by="gene_id")
                self.gene_name = [i for i in record_df.gene_name][0]
                self.record_df = record_df
                self.index = int([i for i in record_df.index][0])
            else:
                record_df = get_gene_record(self.gene_name,by="gene_name")
                self.gene_id = [i for i in record_df.gene_id][0]
                self.record_df = record_df
                self.index = int([i for i in record_df.index][0])
    def get_cds_record(self, source="feature_db_part2.txt"):
        cds_db = pd.read_table(source)
        cds_records = cds_db[cds_db.gene_name == self.gene_name]
        return cds_records
    
    def get_longest_cds(self, source ="feature_db_part2.txt"):
        cds_records = self.get_cds_record(source=source)
        index_longest_cds = cds_records.length.idxmax()
        return cds_records.loc[index_longest_cds,:]
    
    def get_neighbor_records(self,source="feature_db_part1.txt"):
        gene_db = pd.read_table(source)
        total_row =  gene_db.shape[0]
        row_index = self.index
        if row_index >9 and row_index < total_row-9:
            df_left = gene_db.iloc[row_index-10:row_index,:]
            df_right= gene_db.iloc[row_index+1:row_index+11,:]
            return([df_left,df_right])
        elif row_index <=9 :
            df_left= gene_db.iloc[0:row_index,:]
            df_right= gene_db.iloc[row_index+1:21,:]
            return([df_left,df_right])
        else:
            df_left=gene_db.iloc[total_row-21:row_index,:]
            df_right = gene_db.iloc[row_index+1:total_row+1,:]
            return ([df_left,df_right])
    def get_cds_fasta (self,fasta_source = "dmel-all-CDS-r6.26.fasta"):
        cds_name = self.get_longest_cds()["CDS_name"]
        with open(fasta_source) as fh:
            fasta_records = fa_parser(fh)
            for seq_id,seq in fasta_records:
                space_index = seq_id.index(" ")
                seq_name = seq_id[:space_index]
                if seq_name == cds_name:
                    return "\n".join([">"+seq_name,seq])   

In [7]:
test=Gene(gene_name="CG18609")
test.get_cds_fasta()

'>CG18609-PA\nATGCTCCGATACTTGCGCATACCTCAAGCGGATCCTAACCCAATTCCGCTGGCTGGATCACCATGGCCTATCACACTGATTTTGATAGCATATCTGTTGTTTGTCCTTAAATTGGGCAAGATCTTTATGAGAAACCGGAAACCATATGATTTGAAAACGGTCTTGAAGGTCTACAATCTATTTCAGGTGCTATACAATGGTCTCTACTTCGGAATGGTTTTTTATTATCTCTTCATCGTGGGCATTTGCAATCTGCACTGCATAGAAAGCTTTCCCGAGGGCCATGAACGCAAACAATTGGAACGAGTATTGCATGCCGCATATCTGCTGAACAAGGTCCTCGATCTTATGGATACGGTGTTCTTTGTGCTGCGAAAGAGCTATAAGCAGATCACCTTCCTGCACATATATCACCACGTGTTCATGTCCTTTGGAAGTTATGCCCTAACCCGTTACTATGGAACTGGAGGCCATGTCAATGCCGTTGGACTGCTGAACTCCTTGGTGCACACGGTCATGTATTTCTACTACTTTCTGTCTTCAGAATATCCCGGAGTGAGGGCCAATATCTGGTGGAAGAAGTATATAACATTGACACAGCTCTGCCAGTTCTTCATGCTGCTCAGTTATGCCATCTATGTGCGATTCTTTTCACCGAATTGTGGCGTTCCACGCGGTCTTCTCTATCTAAATATGGTGCAAGGCGTTGTGTTCATTTATCTGTTTGGTAAATTCTATATCGACAACTATCTGAGACCTCCGAAGGCGAAAATCAACGCAAAGCAATCGTAG'

In [10]:
test=Gene(gene_name="Nep3")
test.get_cds_fasta()

'>Nep3-PA\nATGACGCGCTACAAGCAGACCGAATTCACGGAGGACGACTCGAGTTCCATCGGCGGCATTCAATTGAACGAAGCGACCGGCCATACGGGCATGCAGATACGCTACCACACCGCCCGGGCTACATGGAATTGGCGCTCGAGAAACAAGACGGAGAAATGGCTGCTCATCACGACTTTTGTGATGGCCATCACGATCTTTACGCTGCTCATCGTCCTTTTCACAGATGGAGGCAGCAGCGATGCGACCAAGCACGTCCTTCACGTCCAGCCGCACCAAAAAGATTGTCCTTCCGGCAATGAGCTTCCTTGCTTGAACAAGCACTGCATCTTCGCCTCGAGCGAGATCCTTAAATCGATCGATGTGACCGTGGATCCGTGCGATGACTTCTATGGATATTCCTGCAATCAATGGATCAAGAACAACCCCATTCCCGAGGGGAAATCCACGTGGGGTACGTTCGGCAAGCTGGAGCAGATGAACCAGTTGATAATACGTAACGTACTGGAGAAGCCGGCAAAAAGCTTTAAGTCGGACGCGGAGCGGAAGGCTAAGGTTTACTATGAATCATGCCTGGATGCGGATGAGCACATGGAGAAGTTGGGGGCAAAGCCCATGAACGATCTACTGCTGCAGATCGGAGGATGGAACGTGACCAAGAGCGGCTACAACGTGGCCAACTGGACGATGGGACACACTCTAAAGATTCTTCATAACAAGTACAACTTCAACTGCCTGTTTGGCTGGGCGATCGGTGAGGATGACAAGAACTCCTCGCGTCACGTCATCCAGATCGATCAGGGCGGTCTGACGCTGCCCACCGCCGACTACTACAACAACAAGACGGACAATCACCGCAAGGTGCTCAACGAGTACATTGAGTACATGACCAAGGTGTGCGTCCTGCTCGGGGCCAACGAATCGGATGCCCGCGCTCAGATGATCGGCGTCATTAATTTCGAGAAGAAGCTGGCCAATATCACCATTCCGCT

In [11]:
# LEFT FLANK
test.get_neighbor_records()[0]

,gene_id,chrom,start,end,number_of_cds,gene_name,name_alias
12959,FBgn0267043,X,19914482,19915281,2,CG45487,NaN
12960,FBgn0267044,X,19919641,19920435,2,CG45488,NaN
12961,FBgn0267045,X,19928416,19929178,2,CG45489,NaN
12962,FBgn0267046,X,19933541,19934294,2,CG45490,NaN
12963,FBgn0267047,X,19938232,19938994,2,CG45491,NaN
12964,FBgn0267048,X,19943357,19944147,2,CG45492,NaN
12965,FBgn0267049,X,19948079,19948873,2,CG45493,NaN
12966,FBgn0267050,X,19952814,19953608,2,CG45494,NaN
12967,FBgn0264309,X,19957548,19958342,2,CG43784,CR43784
12968,FBgn0031080,X,19958393,19959403,2,CG12655,NaN


In [ ]:
# RIGHT FLANK
test.get_neighbor_records()[1]

In [ ]:
test.record_df